In [2]:
# imports
from keras.preprocessing.image import ImageDataGenerator, array_to_img,img_to_array,load_img
import tensorflow.keras as K
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import glob

In [1]:
# get directories
main_direc = os.getcwd()
images_dir = os.path.join(main_direc, 'data/images/images')

# csv location
artist_csv_loc = os.path.join(main_direc, 'data/artists.csv')

In [12]:
# loading data

"""
Hyperparams for data
"""
IMG_WIDTH = 277
IMG_HEIGHT = 277
BATCH_SIZE = 64
NUM_ARTISTS = 11 # this is 11 to get to 10 classes, can be changed...

artists = pd.read_csv(artist_csv_loc)

# Creating a dataframe with the top 10 artists by number of paintings
artists_sort = artists.sort_values(by=['paintings'], ascending=False)

artists_top = artists_sort.head(NUM_ARTISTS)
print(artists_top)

# Images
artists_dir = os.listdir(images_dir) # Files are named after each artists

# Images DataFrame
artists_top_name = artists_top['name'].str.replace(' ', '_').values

images_df = pd.DataFrame()
for name in artists_top_name:
    images_df = pd.concat([images_df, pd.DataFrame(data={'Path': glob.glob('data/images/images/' + name + '/*'), 'Name': name})], ignore_index=True)

print(images_df)

# Create Generator

train_df = images_df.sample(frac=0.8, random_state=200)
test_df = images_df.drop(train_df.index)

if K.backend.image_data_format() == 'channels_first':
    input_shape = (3, IMG_WIDTH, IMG_HEIGHT)
else:
    input_shape = (IMG_WIDTH, IMG_HEIGHT, 3)

# instantiate neural network

# Train

train_generator = ImageDataGenerator(rescale=1.0 / 255,
                                    rotation_range=20,
                                    zoom_range=0.05,
                                    width_shift_range=0.05,
                                    height_shift_range=0.05,
                                    shear_range=0.05,
                                    horizontal_flip=True,
                                    fill_mode="nearest",
                                    validation_split=0.15,
                                    preprocessing_function=preprocess_input
                                    )

test_generator = ImageDataGenerator(rescale=1.0 / 255, preprocessing_function=preprocess_input)

train_gen = train_generator.flow_from_dataframe(
    train_df,
    shuffle=True,
    x_col='Path',
    y_col='Name',
    class_mode='categorical',
    subset="training",
    batch_size=BATCH_SIZE,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    seed=42
)

valid_gen = train_generator.flow_from_dataframe(
    train_df,
    subset="validation",
    shuffle=True,
    x_col='Path',
    y_col='Name',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    target_size=(IMG_WIDTH, IMG_HEIGHT),
    seed=42
)

test_gen = test_generator.flow_from_dataframe(
    test_df,
    x_col='Path',
    batch_size=1,
    shuffle=False,
    class_mode=None,
    target_size=(IMG_WIDTH, IMG_HEIGHT)
)

    id                   name        years                         genre  \
8    8       Vincent van Gogh  1853 – 1890            Post-Impressionism   
30  30            Edgar Degas  1834 - 1917                 Impressionism   
13  13          Pablo Picasso  1881 - 1973                        Cubism   
15  15  Pierre-Auguste Renoir  1841 - 1919                 Impressionism   
19  19         Albrecht Dürer  1471 - 1528          Northern Renaissance   
46  46           Paul Gauguin  1848 – 1903  Symbolism,Post-Impressionism   
16  16         Francisco Goya  1746 - 1828                   Romanticism   
31  31              Rembrandt  1606 - 1669                       Baroque   
20  20          Alfred Sisley  1839 - 1899                 Impressionism   
32  32                 Titian  1488 - 1576    High Renaissance,Mannerism   
22  22           Marc Chagall  1887 - 1985                   Primitivism   

                 nationality  \
8                      Dutch   
30                    F

In [ ]:
# create a simple baseline model
